---
title: "Mental Health"
author: 
  - "Lauro Reyes Rosas - 214532"
  - "Blanca Estela García Manjarrez - 118886"
  - "Yuneri Pérez Arellano - 199813"
format:
  html:
    code-fold: false
jupyter: python3
subtitle: "ITAM - Aprendizaje de Máquina"
date: "Fecha de entrega: 2024-12-11"
professor: "Felipe González"
cache: true
---


En este proyecto, abordaremos la tarea de predecir la variable objetivo **Depresión** utilizando un conjunto de datos titulado **Salud Mental**[link][https://www.kaggle.com/competitions/playground-series-s4e11/data]. Este dataset es el resultado de una encuesta integral cuyo propósito fue analizar los factores asociados al riesgo de depresión en adultos. La comprensión de estos factores puede contribuir al desarrollo de intervenciones más efectivas en el ámbito de la salud mental.

La encuesta fue llevada a cabo de manera anónima entre enero y junio de 2023 en diversas ciudades, involucrando a personas de diferentes orígenes y profesiones. Los participantes, con edades entre 18 y 60 años, compartieron voluntariamente información sobre una amplia variedad de aspectos, tales como edad, género, ciudad de residencia, entre otros.

El conjunto de datos contiene 140,700 registros en el conjunto de entrenamiento y está compuesto por 20 variables, descritas a continuación:

| Variable                                  | Descripción                                               |
| ----------------------------------------- | --------------------------------------------------------- |
| id                                        | Identificador único del registro                          |
| Nombre                                    | Nombre del participante                                   |
| Género                                    | Género del participante                                   |
| Edad                                      | Edad del participante                                     |
| Ciudad                                    | Ciudad de residencia                                      |
| Profesionista/Estudiante                  | Ocupación principal                                       |
| Profesión                                 | Campo laboral o académico                                 |
| Presión\_académica                        | Nivel de presión relacionado con los estudios             |
| Presión\_laboral                          | Nivel de presión relacionado con el trabajo               |
| CGPA                                      | Promedio general acumulado                                |
| Satisfacción\_académica                   | Nivel de satisfacción con los estudios                    |
| Satisfacción\_laboral                     | Nivel de satisfacción con el trabajo                      |
| Tiempo\_dormir                            | Horas promedio de sueño por día                           |
| Hábitos\_alimenticios                     | Calidad percibida de los hábitos alimenticios             |
| Grado\_académico                          | Nivel más alto de educación alcanzado                     |
| Pensamientos\_suicidas                    | Indicadores de pensamientos suicidas                      |
| Horas\_estudio/trabajo                    | Promedio de horas dedicadas a estudio o trabajo           |
| Estrés\_financiero                        | Percepción de presión financiera                          |
| Antecedente\_familiar\_enfermedad\_mental | Presencia de antecedentes familiares de enfermedad mental |
| Depresión                                 | Indicador binario de depresión (variable objetivo)        |


El análisis de este conjunto de datos permitirá explorar y modelar las relaciones entre las variables predictoras y la variable objetivo, contribuyendo así a un mejor entendimiento del impacto de diversos factores en la salud mental.

## EDA

In [ ]:
#| echo: false
#| label: librerias
import numpy as np
import pandas as pd 
import matplotlib.pylab as plt 
import seaborn as sns 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

sns.set_palette("pastel")

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| echo: false
#| label: info
train = pd.read_csv('data/train.csv')
test = pd.read_csv("data/test.csv")
# rename columns
train.columns = [col.lower().replace(" ","_") for col in train.columns]
test.columns = [col.lower().replace(" ","_") for col in test.columns]
train.info()

El conjunto de entrenamiento consta de 140,700 registros y 20 columnas.

**Variables numéricas**

In [ ]:
#| echo: false
train.describe().round(2).style.format(precision=2)

El análisis exploratorio de las variables numéricas revela que las relacionadas con satisfacción, presión y estrés están en una escala de 1 a 5, con medias cercanas a 3. La edad de los participantes oscila entre 18 y 60 años, con una media de 40.4, mientras que las horas de trabajo/estudio tienen una media de 6.25 horas diarias. El rendimiento académico (CGPA) muestra valores altos, con una media de 7.66. La variable objetivo, depression, indica que alrededor del 18% de los participantes presenta riesgo de depresión.


**Variables categóricas**

In [ ]:
#| echo: false
train.describe(include="object")

En cuanto a las variables categóricas, destacan 98 ciudades únicas y 64 profesiones, con "Teacher" como la más común. Duraciones de sueño como "Less than 5 hours" son las más reportadas, mientras que los antecedentes familiares de enfermedad mental y pensamientos suicidas tienen predominancia de respuestas negativas. Estos resultados iniciales subrayan la diversidad del dataset y señalan áreas clave para el tratamiento de datos, como los valores nulos y categorías con alta cardinalidad.


#### Variable Objetivo

In [ ]:
#| echo: false
#| label: target variable dist
target_colors = [
    "#5EDB92",
    "#5A665F",
]
plt.figure(figsize=(6, 6))
plt.pie(
    train["depression"].value_counts(),
    labels=train["depression"].value_counts().index,
    textprops={"fontsize": 15, "color": "black"},
    colors=target_colors,
    autopct="%.0f%%",
    explode=[0.03, 0.03],
)
plt.title("Distribución de la variable Depresión", fontsize=14)
plt.show()

El análisis de la variable objetivo depression revela un conjunto de datos desbalanceado, donde aproximadamente el 82% de los registros corresponden a individuos sin depresión y solo el 18% a casos positivos. Esta desigualdad es visualmente evidente en la gráfica de distribución.

El desbalance en la variable objetivo establece un desafío en el modelado predictivo, ya que los modelos pueden inclinarse hacia la clase mayoritaria. Sin embargo, también sugiere que un punto de referencia inicial para la métrica de precisión (accuracy) debe superar el 82%, ya que ese sería el desempeño esperado al predecir siempre la clase mayoritaria.


##### Revisión de valores nulos


In [ ]:
#| echo: false
#| label: NA check

null_percentage = (train.isnull().sum() / len(train)) * 100
non_null_percentage = 100 - null_percentage
percentage_df = pd.DataFrame({
    'Valores Nulos (%)': null_percentage,
    'Valores Presentes (%)': non_null_percentage
})
# Filtrar solo columnas con valores nulos
percentage_df = percentage_df[null_percentage > 0]
# Gráfico apilado
percentage_df.plot(kind='bar', stacked=True, figsize=(8, 6), color=['red', 'green'])
plt.title("Porcentaje de Valores Nulos")
plt.ylabel("Porcentaje (%)")
plt.xticks(rotation=45)
plt.legend(loc='upper right')
plt.show()

En particular, las variables relacionadas con estudiantes, como academic_pressure, cgpa, y study_satisfaction, presentan hasta un 80% de valores nulos. Este resultado refleja una menor representación de estudiantes en comparación con profesionistas dentro del dataset.

A pesar del alto porcentaje de valores faltantes en estas variables, no se eliminarán del análisis, ya que contienen información valiosa para los casos específicos de estudiantes. Este enfoque permite preservar datos que pueden ser relevantes en el desarrollo del modelo predictivo, especialmente para evaluar las diferencias entre grupos ocupacionales. Las estrategias de imputación y tratamiento de valores nulos serán fundamentales para garantizar la calidad del análisis posterior.


#### Ingenieria de Variables
Se añadieron características derivadas para enriquecer el dataset y mejorar el análisis. Estas incluyen:

* satisfaction_by_work: Relación entre presión laboral y satisfacción en el trabajo.
* satisfaction_by_study: Relación entre presión y satisfacción académica.
* age_work_satisfaction: Razón entre edad y satisfacción laboral.
* cgpa_study: Relación entre promedio académico y presión académica.

Estas nuevas variables buscan capturar interacciones clave, como el impacto de la presión en la satisfacción o el desempeño bajo diferentes condiciones, y tienen el potencial de mejorar el modelo predictivo. Las varibales se crean con la siguiente función:

In [ ]:
#| label: new feats
def new_feats(df):
    df = (
        df.assign(satisfaction_by_work=df["work_pressure"] / df["job_satisfaction"])
        .assign(
            satisfaction_by_study=df["academic_pressure"] / df["study_satisfaction"]
        )
        .assign(age_work_satisfaction=df["age"] / df["job_satisfaction"])
        .assign(cgpa_study=df["cgpa"] / df["academic_pressure"])
    )
    return df

train = new_feats(train).copy()
test = new_feats(test).copy()

Dado que algunas variables categóricas presentan un gran número de categorías únicas, se implementó una estrategia para agrupar las categorías menos representativas en una etiqueta denominada "Other". Esto asegura que las categorías seleccionadas representen al menos el 90% de los datos, reduciendo la dimensionalidad y facilitando el análisis.

Las columnas ajustadas incluyen:

* city
* profession
* sleep_duration
* dietary_habits
* degree

Esta transformación permite simplificar la estructura del dataset sin perder información relevante, mejorando la manejabilidad del modelo y reduciendo el riesgo de sobreajuste debido a categorías con baja frecuencia.


In [ ]:
#| echo: false

num_feats = train.select_dtypes(include="float64").columns
obj_feats = train.select_dtypes(include="object").columns
target = train.select_dtypes(include="int64").columns

def group_low_frequency(train, test, column, threshold_percentage):
    """
    Agrupa las categorías con representación acumulativa menor al umbral definido en "Other".

    Args:
        train (pd.DataFrame): DataFrame de entrenamiento.
        test (pd.DataFrame): DataFrame de prueba.
        column (str): Columna objetivo a transformar.
        threshold_percentage (float): Umbral de porcentaje (0-100) para incluir categorías acumulativas.

    Returns:
        pd.Series, pd.Series: Columnas transformadas para `train` y `test`.
    """
    # Calcular la frecuencia normalizada y ordenarla
    value_counts = train[column].value_counts(normalize=True).sort_values(ascending=False)

    # Calcular el porcentaje acumulativo
    cumsum = value_counts.cumsum()

    # Identificar categorías que generan el porcentaje acumulativo dentro del umbral
    valid_categories = cumsum[cumsum <= threshold_percentage].index

    # Transformar la columna en train
    train_transformed = train[column].apply(lambda x: x if x in valid_categories else "Other")

    # Transformar la columna en test (usar categorías válidas de train)
    test_transformed = test[column].apply(lambda x: x if x in valid_categories else "Other")

    return train_transformed, test_transformed

In [ ]:
train["city"], test["city"] = group_low_frequency(train, test, 'city', 0.9)
train["profession"], test["profession"] = group_low_frequency(train, test, 'profession', 0.9)
train["sleep_duration"], test["sleep_duration"] = group_low_frequency(train, test, 'sleep_duration', 0.9)
train["dietary_habits"], test["dietary_habits"] = group_low_frequency(train, test, 'dietary_habits', 0.9)
train["degree"], test["degree"] = group_low_frequency(train, test, 'degree', 0.9)

Tras la agrupación, las variables categóricas principales quedan resumidas con las categorías más representativas

In [ ]:
#| echo: false
train[["city","profession","sleep_duration","dietary_habits","degree"]].describe(include="object")

##### Relación variables categóricas y Depresión


In [ ]:
#| echo: false
#| label: catplt
def catplt(x_var, aspect = 4):
    gs = sns.catplot(
        data=train,
        x=x_var,
        kind="count",
        hue="depression",
        height=2,
        aspect=aspect,
        sharey=False,
        palette=target_colors,
        legend_out=False,
        legend="auto",
        width=0.5,
    )
    for ax in gs.axes.ravel():
        for i in ax.containers:
            ax.bar_label(i, label_type="edge", fontsize=8)
            ax.set_xlabel("", fontsize=12)
            ax.set_ylabel("", fontsize=12)
            ax.tick_params(axis="both", labelsize=8)
            gs._legend.remove()
            gs.set_titles(size=8)

    plt.legend(title="Target", title_fontsize=8, fontsize=8, loc="best")
    plt.show()

In [ ]:
#| echo: false
catplt(x_var="work/study_hours", aspect= 8)

El análisis gráfico de la variable work/study_hours muestra un patrón interesante: a medida que aumentan las horas dedicadas al trabajo o estudio, también aumenta la proporción de casos con indicadores de depresión.


In [ ]:
#| echo: false
catplt(x_var="working_professional_or_student")

Los resultados muestran que más de la mitad de los estudiantes en el dataset tienen indicadores de depresión. Esto resalta una diferencia notable en comparación con los profesionales, sugiriendo que el entorno académico puede ser un factor de estrés significativo.


In [ ]:
#| echo: false
catplt(x_var="gender")

Aunque hay más hombres con depresión en términos absolutos, los porcentajes de depresión entre hombres y mujeres son similares, alrededor del 18%. Esto indica que la distribución de la variable objetivo no presenta un sesgo significativo entre géneros.

In [ ]:
#| echo: false
catplt(x_var="study_satisfaction")

Se observa una clara relación inversa entre la satisfacción en los estudios y la propensión a la depresión. A menor satisfacción académica, mayor es la proporción de casos positivos de depresión, lo que refuerza la importancia del bienestar en el ámbito educativo.


##### Mapa de correlación entre variable numéricas

In [ ]:
#| echo: false
plt.style.use("default")
corr_mat = train[num_feats].corr()
mask = np.triu(np.ones_like(corr_mat, dtype=bool))
cmap = sns.diverging_palette(230, 30, as_cmap=True)
f, ax = plt.subplots(figsize=(9, 7))
sns.heatmap(
    corr_mat,
    mask=mask,
    cbar=True,
    cmap=cmap,
    center=0,
    square=False,
    annot=False,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
)
plt.title("Correlation Matrix\n")
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
plt.show()

## Modelos

In [ ]:
#| echo: false
#| label: Sklearn libraries

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)

from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import LogisticRegression

### CatBoost

CatBoost es un algoritmo de aprendizaje automático basado en árboles de decisión que utiliza técnicas avanzadas de boosting. Es especialmente útil para trabajar con datos categóricos, ya que implementa un manejo eficiente de estas variables mediante codificación automática, lo que reduce la necesidad de preprocesamiento adicional. Además, CatBoost es conocido por su capacidad de minimizar el riesgo de sobreajuste y su alto desempeño en tareas de clasificación y regresión.

Elegimos CatBoost para este ejercicio debido a la naturaleza del dataset, que contiene una combinación de variables numéricas y categóricas. Su capacidad para procesar variables categóricas de forma nativa, junto con su robustez frente al desbalance de datos, lo hace ideal para predecir nuestra variable objetivo, **depression**, en un conjunto de datos con estructuras y distribuciones complejas. Además, el rendimiento competitivo de CatBoost en comparación con otros algoritmos de boosting lo convierte en una opción sólida para este tipo de problema.

Para entrenar el modelo, el dataset se preprocesó llenando valores faltantes con "0" y ajustando los tipos de datos según la naturaleza de las variables (numéricas, categóricas y objetivo). Posteriormente, se dividieron los datos en conjuntos de entrenamiento y prueba, utilizando el 70% de los datos para entrenamiento y el 30% para prueba. Este procedimiento garantiza una evaluación justa del modelo, preservando datos no vistos para validar su desempeño y evitar el sobreajuste.


In [ ]:
#| echo: false
#| label: Catboost
cat_train = train.fillna("0")
cat_test = test.fillna("0")
cat_train[num_feats] = cat_train[num_feats].astype("float64")
cat_train[obj_feats] = cat_train[obj_feats].astype("string")
cat_train[target] = cat_train[target].astype("int64")
cat_test[num_feats] = cat_test[num_feats].astype("float64")
cat_test[obj_feats] = cat_test[obj_feats].astype("string")
y = cat_train["depression"]
X = cat_train.drop(["depression"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=4
)

print("----------X Shape---------")
print(X.shape)
print("*************************************")
print("----------y Shape------------")
print(y.shape)

En este paso, utilizamos Optuna, una biblioteca eficiente para la optimización de hiperparámetros, con el objetivo de encontrar la mejor configuración para el modelo de CatBoostClassifier. La función de optimización busca maximizar el desempeño del modelo en términos de precisión (accuracy) utilizando un enfoque basado en validación cruzada.
Espacio de Búsqueda

El espacio de búsqueda incluye los siguientes hiperparámetros:

In [ ]:
def objective(trial):
    # Espacio de búsqueda para los hiperparámetros
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 20.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
    }

    # Modelo de CatBoost
    model = CatBoostClassifier(**params, cat_features=list(obj_feats), verbose=False)

    # Evaluación usando validación cruzada
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score

Función Objetivo

La función objetivo se define como el promedio de la precisión obtenida mediante validación cruzada con 3 particiones ($k=3$):

$$\text{Score} = \frac{1}{k} \sum_{i=1}^{k} \text{Accuracy}(i)$$

Donde $\text{Accuracy}(i)$ es la precisión en el $i$-ésimo fold de la validación cruzada. El modelo de CatBoost se entrena y evalúa en cada iteración con una combinación específica de hiperparámetros sugerida por Optuna.


In [ ]:
best_params = {'iterations': 633,
 'learning_rate': 0.18918646362839026,
 'depth': 4,
 'colsample_bylevel': 0.9909843814690833,
 'subsample': 0.9778634549619083,
 'min_data_in_leaf': 1,
 'l2_leaf_reg': 10.504885456201343,
 'random_strength': 0.664861942866057}

In [ ]:
#| eval: false
#| label: Catboost run
cat_interp = CatBoostClassifier(**best_params, verbose=True, cat_features=X[obj_feats].columns.values, early_stopping_rounds=200)
cat_cv_last = StratifiedKFold(20, shuffle=True, random_state=42)
cat_cv_splits = cat_cv_last.split(X, y)
acc_scores = []
last_test_df_preds = []
cat_features = X[obj_feats].columns.values
X[cat_features] = X[cat_features].fillna('nan').astype(str)
X_test_df_pool = Pool(test, cat_features=X[obj_feats].columns.values)
for i, (full_train_idx, valid_idx) in enumerate(cat_cv_splits):
    model_fold = CatBoostClassifier(**best_params)
    X_train_fold, X_valid_fold = X.loc[full_train_idx], X.loc[valid_idx]
    y_train_fold, y_valid_fold = y.loc[full_train_idx], y.loc[valid_idx]
    X_train_pool = Pool(X_train_fold, y_train_fold, cat_features=X[obj_feats].columns.values)
    X_valid_pool = Pool(X_valid_fold, y_valid_fold, cat_features=X[obj_feats].columns.values)
    model_fold.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=False, early_stopping_rounds=100)
    valid_pred = model_fold.predict(X_valid_pool)
    score = accuracy_score(y_valid_fold, valid_pred)
    acc_scores.append(score)
    test_df_pred = model_fold.predict_proba(X_test_df_pool)[:, 1]
    last_test_df_preds.append(test_df_pred)
    print(f'Fold {i + 1} Accuracy Score: {score}')

Fold 1 Accuracy Score: 0.9383

Fold 2 Accuracy Score: 0.9420

Fold 3 Accuracy Score: 0.9375

Fold 4 Accuracy Score: 0.9389

Fold 5 Accuracy Score: 0.9419

Fold 6 Accuracy Score: 0.9407

Fold 7 Accuracy Score: 0.9396

Fold 8 Accuracy Score: 0.9429

Fold 9 Accuracy Score: 0.9403

Fold 10 Accuracy Score: 0.9395

In [ ]:
#| eval: false
from sklearn.metrics import classification_report
print(classification_report(y_valid_fold, valid_pred))